# Importing necessary libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
import cv2
import mediapipe as mp
import time
from scipy import signal
import scipy

# Functions for filtering the signal

In [2]:
def filtering(data,fs,lowerCutoff=1, higherCutoff=3, filterOrder=5):
    lowerCutoffDigital = lowerCutoff / (0.5 * fs)
    higherCutoffDigital = higherCutoff / (0.5 * fs)
    b, a = signal.butter(filterOrder, [lowerCutoffDigital, higherCutoffDigital], btype='band', analog=False)
    filtsignal = signal.filtfilt(b, a, data)
    return filtsignal
def bandpass_filter(data,fps, lowcut, highcut):
    fs = fps # Частота дискретизации (количество измерений сигнала в 1 сек)
    nyq = 0.5 * fs # Частота Найквиста
    low = float(lowcut) / float(nyq)
    high = float(highcut) / float(nyq)
    order = 6.0 # Номер фильтра в scipy.signal.butter
    b, a = butter(order, [low, high], btype='band')
    bandpass = lfilter(b, a, data)
    return bandpass

# Function for finding the heart rate of a rPPG Signal

In [3]:
def HR_FINDER(wave,fs):
    fftData = np.fft.fft(wave,1000)[0:500]
    hz=np.linspace(0,fs/2,int(len(fftData)))
    powerSpectrum = np.abs(fftData)**2
    maxFreq = np.argmax(powerSpectrum)
    HR=hz[maxFreq]*60
    return HR

In [4]:
def fft_wave(wave):
    fftData = np.fft.fft(wave,1000)[0:500]
    powerSpectrum = np.abs(fftData)**2
    return powerSpectrum

# Function for face Masking

In [5]:
#img=cv2.imread(r"C:\Users\Lenovo\Pictures\Camera Roll\WIN_20220519_14_51_19_")
#img = cv2.imread(r"C:\Users\Lenovo\Pictures\Camera Roll\WIN_20220520_20_11_49_Pro.jpg")
# img = cv2.imread(r"C:\Users\Lenovo\Pictures\Camera Roll\WIN_20220520_20_16_00_Pro.jpg")
#v=cv2.VideoCapture(r"C:\Users\Lenovo\Desktop\Camera Roll\WIN_20220325_17_44_26_Pro.mp4")
mpDraw = mp.solutions.drawing_utils
mpFaceMesh = mp.solutions.face_mesh
faceMesh = mpFaceMesh.FaceMesh(max_num_faces=2,min_detection_confidence=0.5)
drawSpec = mpDraw.DrawingSpec(thickness=1, circle_radius=2,color=(0,128,0))
def masking(img):
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = faceMesh.process(imgRGB)
    pTime=0
    face=[]
    num1=[67,109,10,338,104,69,108,151,337,299]
    num2=[66,280,425,426,266,50,205,147,187,207,432,138]
    facenum=[21,54,143,67,149,14,338,297,332,284,251,389,356,454,323,361,288,397,365,379,378,400,377,152,148,176,149,150,136,172
            ,58,132,43,234,127,162]
    facecont=[]
    faceadj=[33,21,3,11,29,1]
    left=[]
    right=[]
    if results.multi_face_landmarks:
        landmarkz=results.multi_face_landmarks[0]
        for faceLms in results.multi_face_landmarks:
            #mpDraw.draw_landmarks(img, faceLms, mpFaceMesh.FACEMESH_CONTOURS,
                                      #drawSpec,drawSpec)
            #print(frame.shape)
            for id,lm in enumerate(faceLms.landmark):
                ih, iw, ic = img.shape
                x,y = int(lm.x*iw), int(lm.y*ih)
                #cv2.putText(img,str(id),(x,y),cv2.FONT_HERSHEY_PLAIN,.5,(0,0,255),1)
                face.append((x,y))
                if id in facenum:
                    facecont.append([x,y])
                if id ==107 or id==55:
                    right.append((x,y))
                if id ==336 or id==285:
                    left.append((x,y))
                #if id in num1:
                    #cv2.rectangle(img,(face[id][0]-40,face[id][1]-40),(face[id][0]+40,face[id][1]+40),(255,0,0),3)
                #if id in num2:
                #cv2.rectangle(img,(face[id][0]-20,face[id][1]-40),(face[id][0]+20,face[id][1]+40),(255,0,0),3)

        ###################################### face_oval#######################################################
        blobs=[]
        l = []
        i=0
        for source_id,target_id in mpFaceMesh.FACEMESH_FACE_OVAL:
            if i in faceadj:
                source=landmarkz.landmark[source_id]
                real_source=(int(source.x*img.shape[1]),int(source.y*img.shape[0]))
                l.append([(int(source.x*img.shape[1])),int(source.y*img.shape[0]-20)])
                blobs.append(l)
            else:
                source=landmarkz.landmark[source_id]
                real_source=(int(source.x*img.shape[1]),int(source.y*img.shape[0]))
                l.append([(int(source.x*img.shape[1])),int(source.y*img.shape[0])])
                blobs.append(l)
            i+=1
        coords = blobs[0]
        for i in range(0,2):
            coords.append([left[i][0],left[i][1]])
        req_coord_lefteyebrow=[4,0,2,7,1,6,5,3,8,9]
        sel_coords = []

        # req_coords = [11,27,9,22,7,28,24,14,6,0,15,5,26,35,19,33,2]
        req_coords = [11,29,1,27,13,9,12,4,22,17,7,31,28,23,24,34,14,16,6,25,0,30,2,15,20,5,10,26,32,35,8,19,18,33,21,3]
        for i in req_coords:
            sel_coords.append(coords[i])
        mas = np.zeros((img.shape[0],img.shape[1],3),dtype=np.uint8)
        cv2.fillPoly(mas, pts=[np.asarray(sel_coords)], color=(255,255,255))
        masked = cv2.bitwise_and(img, mas)

        ######################################LIPS###############################################################
        blobs=[]
        l = []
        i=0
        
        for source_id,target_id in mpFaceMesh.FACEMESH_LIPS:
            source=landmarkz.landmark[source_id]
            target=landmarkz.landmark[target_id]
            real_source=(int(source.x*img.shape[1]),int(source.y*img.shape[0]))
            real_target=(int(target.x*img.shape[1]),int(target.y*img.shape[0]))
            l.append([(int(source.x*img.shape[1])),int(source.y*img.shape[0])])
            blobs.append(l)
        coords = blobs[0]
        sel_coords = []
        # req_coords = [11,27,9,22,7,28,24,14,6,0,15,5,26,35,19,33,2]
        req_coords = [11,29,1,27,13,9,12,4,22,17,7,31,28,23,24,34,14,16,6,25,0,30,2,15,20,5,10,26,32,35,8,19,18,33,21,3]
        req_coord_eye=[33,32,4,14,18,6,0,26,39,7,23,16,11,5,38,3,27,22]
        for i in req_coord_eye:
            sel_coords.append(coords[i])
        mas = np.zeros((img.shape[0],img.shape[1],3),dtype=np.uint8)
        cv2.fillPoly(mas, pts=[np.asarray(sel_coords)], color=(255,255,255))
        masked1 = cv2.bitwise_and(masked, mas)
        lip=cv2.subtract(masked,masked1)
        ################################lefteye####################################################################
        blobs=[]
        l = []
        i=0
        for source_id,target_id in mpFaceMesh.FACEMESH_LEFT_EYE:
            source=landmarkz.landmark[source_id]
            target=landmarkz.landmark[target_id]
            real_source=(int(source.x*img.shape[1]),int(source.y*img.shape[0]))
            real_target=(int(target.x*img.shape[1]),int(target.y*img.shape[0]))
            l.append([(int(source.x*img.shape[1])),int(source.y*img.shape[0])])
            blobs.append(l)
        coords = blobs[0]
        sel_coords = []
        #ctr = np.array(facecont).reshape((-1,1,2)).astype(np.int32)
        #cv2.drawContours(img,[ctr],0,(255,255,255),1)
        req_coord_lefteye=[3,10,6,5,12,4,0,13,7,8]
        for i in req_coord_lefteye:
            sel_coords.append(coords[i])
        mas = np.zeros((img.shape[0],img.shape[1],3),dtype=np.uint8)
        cv2.fillPoly(mas, pts=[np.asarray(sel_coords)], color=(255,255,255))
        masked2 = cv2.bitwise_and(lip, mas)
        lefteye=cv2.subtract(lip,masked2)
        ###############################################righteye#############################################################
        blobs=[]
        l = []
        i=0
        for source_id,target_id in mpFaceMesh.FACEMESH_RIGHT_EYE:
            source=landmarkz.landmark[source_id]
            target=landmarkz.landmark[target_id]
            real_source=(int(source.x*img.shape[1]),int(source.y*img.shape[0]))
            real_target=(int(target.x*img.shape[1]),int(target.y*img.shape[0]))
            l.append([(int(source.x*img.shape[1])),int(source.y*img.shape[0])])
            blobs.append(l)
        coords = blobs[0]
        sel_coords = []
        req_coord_righteye=[1,2,3,6,11,5,0,8,4,7,9,12]
        for i in req_coord_righteye:
            sel_coords.append(coords[i])
        mas = np.zeros((img.shape[0],img.shape[1],3),dtype=np.uint8)
        cv2.fillPoly(mas, pts=[np.asarray(sel_coords)], color=(255,255,255))
        masked2 = cv2.bitwise_and(lefteye, mas)
        righteye=cv2.subtract(lefteye,masked2)
        ##################################################lefteyebrow############################################################
        blobs=[]
        l = []
        i=0
        for source_id,target_id in mpFaceMesh.FACEMESH_LEFT_EYEBROW:
            source=landmarkz.landmark[source_id]
            target=landmarkz.landmark[target_id]
            real_source=(int(source.x*img.shape[1]),int(source.y*img.shape[0]))
            real_target=(int(target.x*img.shape[1]),int(target.y*img.shape[0]))
            l.append([(int(source.x*img.shape[1])),int(source.y*img.shape[0])])
            blobs.append(l)
        coords = blobs[0]
        for i in range(0,2):
            coords.append([left[i][0],left[i][1]])
        sel_coords = []
        req_coord_lefteyebrow=[4,0,2,7,1,6,5,3,8,9]
        for i in req_coord_lefteyebrow:
            sel_coords.append(coords[i])
        mas = np.zeros((img.shape[0],img.shape[1],3),dtype=np.uint8)
        cv2.fillPoly(mas, pts=[np.asarray(sel_coords)], color=(255,255,255))
        masked3 = cv2.bitwise_and(righteye, mas)
        lefteyebrow=cv2.subtract(righteye,masked3)
        ##################################################righteyebrow#####################################################3
        blobs=[]
        l = []
        i=0
        for source_id,target_id in mpFaceMesh.FACEMESH_RIGHT_EYEBROW:
            source=landmarkz.landmark[source_id]
            target=landmarkz.landmark[target_id]
            real_source=(int(source.x*img.shape[1]),int(source.y*img.shape[0]))
            real_target=(int(target.x*img.shape[1]),int(target.y*img.shape[0]))
            l.append([(int(source.x*img.shape[1])),int(source.y*img.shape[0])])
            blobs.append(l)
        coords = blobs[0]
        for i in range(0,2):
            coords.append([right[i][0],right[i][1]])
        sel_coords = []
        req_coord_righteyebrow=[5,4,7,1,9,8,6,2,3,0]
        for i in req_coord_righteyebrow:
            sel_coords.append(coords[i])
        mas = np.zeros((img.shape[0],img.shape[1],3),dtype=np.uint8)
        cv2.fillPoly(mas, pts=[np.asarray(sel_coords)], color=(255,255,255))
        masked4 = cv2.bitwise_and(lefteyebrow, mas)
        righteyebrow=cv2.subtract(lefteyebrow,masked4)
    #################################################################################################################3
        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime
        return righteyebrow
    else:
        return img

# Preprocessing of the spatially averged 1d-signal

In [6]:
def RAW(matrix):
    green_matrix=[]
    for z in range(matrix.shape[0]):
        for i in range(3):
            green=matrix[z,:,i]
            greend=scipy.signal.detrend(green) ####detrending
            greenN=greend/np.std(greend)
            greenf=filtering(greenN,fs,.7,3)#### filtering
            greenf=(greenf-greenf.min())/(greenf.max()-greenf.min())
            green_matrix.append(greenf)
    return np.array(green_matrix)

# Main code for saving the data

In [9]:
save_path=r'C:\Users\PhysioSens\Desktop\python-jupyter\MS-Work-Documentation\Novel-HR\Physiosens1D\data\val' #### give the path where you want to save the file
DATA_path=Path(r'D:\DL_DATA\UBFC TEST')#### Data path where the train or test data is 
#num1=[109,10,338,108,151,337]
num1=[10]
num2=[50,280]
mpFaceMesh = mp.solutions.face_mesh
faceMesh = mpFaceMesh.FaceMesh(max_num_faces=1)
for sub in os.listdir(DATA_path):
    full_path=DATA_path/sub
    ext=os.listdir(full_path)
    label_path=full_path/ext[0]
    vid_path=full_path/ext[1]
    vid=cv2.VideoCapture(str(vid_path))
    label=np.loadtxt(label_path)
    fs=vid.get(cv2.CAP_PROP_FPS)
    matrix=np.zeros((len(num1)+len(num2),150,3))
    count=0
    flag=0
    num_sub=0
    for i in range(int(vid.get(cv2.CAP_PROP_FRAME_COUNT))):
        ret,frame=vid.read()
        if ret==False:
            break
        frame=masking(frame)### you can use this if you want to create a mask to remove non-skin pixel highly recommended
        imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = faceMesh.process(imgRGB)
        if i%150==0 and i!=0:
            num_sub+=1
            wave=label[0][i-150:i]
            wave=scipy.signal.detrend(wave)
            wave=(wave-wave.min())/(wave.max()-wave.min())
            hr=label[1][i-150:i]
            sub_label=(wave,hr)
            
            green_matrix=RAW(matrix) #### Preprocessing for the raw signals
            matrix=np.zeros((len(num1)+len(num2),150,3))
            flag,count=0,0


            green_vid_path=os.path.join(save_path,'data')
            green_label_path=os.path.join(save_path,'label')
            os.makedirs(green_vid_path,exist_ok=True)
            os.makedirs(green_label_path,exist_ok=True)
            np.save(os.path.join(green_vid_path,f'{sub} {num_sub}.npy'),green_matrix)####saving the video data
            np.save(os.path.join(green_label_path,f'{sub} {num_sub}.npy'),sub_label)##### saving the ground-truth PPG

            
            
            
            
        if results.multi_face_landmarks:
            for faceLms in results.multi_face_landmarks:
                face=[]
                for id,lm in enumerate(faceLms.landmark):
                    ih, iw, ic = frame.shape
                    x,y = int(lm.x*iw), int(lm.y*ih)
                    face.append((x,y))
                    if id in num1:
                        #cv2.rectangle(frame,(face[id][0]-60,face[id][1]-25),(face[id][0]+60,face[id][1]+25),(255,0,0),3)
                        x=np.mean(frame[face[id][1]-25:face[id][1]+25,face[id][0]-60:face[id][0]+60,2])####Can change the rectangluar corrdinate to
                        ###select the ROI
                        if x>0 and x<255:
                            matrix[flag,count,2]=x
                        y=np.mean(frame[face[id][1]-25:face[id][1]+25,face[id][0]-60:face[id][0]+60,1])
                        if y>0 and y<255:
                            matrix[flag,count,1]=y
                        z=np.mean(frame[face[id][1]-25:face[id][1]+25,face[id][0]-60:face[id][0]+60,0])
                        if z>0 and z<255:
                            matrix[flag,count,0]=z
                        flag+=1
                    if id in num2:
                        #cv2.rectangle(frame,(face[id][0]-25,face[id][1]-25),(face[id][0]+25,face[id][1]+25),(255,0,0),3)
                        matrix[flag,count,2]=np.mean(frame[face[id][1]-25:face[id][1]+25,face[id][0]-25:face[id][0]+25,2])
                        matrix[flag,count,1]=np.mean(frame[face[id][1]-25:face[id][1]+25,face[id][0]-25:face[id][0]+25,1])
                        matrix[flag,count,0]=np.mean(frame[face[id][1]-25:face[id][1]+25,face[id][0]-25:face[id][0]+25,0])
                        ####Can change the rectangluar corrdinate to select the ROI
                        flag+=1
        count+=1
        flag=0
        
        cv2.imshow('frame',frame)#### comment this for not coming the image and can make the datasaving faster
        key=cv2.waitKey(1)&0XFF
        if key==ord('q'):
            break
    cv2.destroyAllWindows()